# Getting started with authentication and package installation

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!earthengine authenticate
import ee
ee.Initialize()

In [ ]:
import subprocess
try:
  import geemap
except ImportError:
  print('geemap package not installed. Installing ...')
  subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

In [ ]:
import geemap
import os
import math

In [ ]:
!pip install rasterio
!pip install rio-cogeo
import rasterio

# Prepare datasets for extracting raster value with buffered points

In [ ]:
# Import datasets; some are handled in GEE so not repeated here
# Alternative Landsat and MODIS datasets may be used depending on the scale and resolution of interests
DAYMET = ee.ImageCollection("NASA/ORNL/DAYMET_V4")
PRISM = ee.ImageCollection("OREGONSTATE/PRISM/AN81d") # Daymet does not have the most recent records yet so PRISM is used in its place for the most recent time period
NDWI = ee.ImageCollection("MODIS/MOD09GA_006_NDWI")
MODISTree = ee.ImageCollection("MODIS/006/MOD44B")
MODISGPP = ee.ImageCollection("MODIS/006/MOD17A2H")
MODISEVI = ee.ImageCollection("MODIS/MOD09GA_006_EVI")
MODISLST = ee.ImageCollection('MODIS/006/MOD11A1')
RAP = ee.ImageCollection("projects/rangeland-analysis-platform/vegetation-cover-v2");
ROI = ee.FeatureCollection("users/site") # Customize the pathway to import shapefile for the site to predict

In [ ]:
roi = ROI.geometry()

In [ ]:
# Climate datasets; for estimation before 2021
precipitation = DAYMET.select("prcp")
airTmin = DAYMET.select("tmin")
airTmax = DAYMET.select("tmax")
vpd = DAYMET.select("vp")

In [ ]:
# Climate datasets from prism (lower resolution than daymet); for estimation in or after 2021
precipitation = PRISM.select("ppt")
Tmean = PRISM.select("tmean")
vpdmin = PRISM.select("vpdmin")
vpdmax = PRISM.select("vpdmax")

# Define cloud mask and temporal interpolation for remote sensing data

In [ ]:
def maskgpp (image):
  modisgppQA = image.select("Psn_QC");
  def extractQABits (qaBand, bitStart, bitEnd):
    numBits = bitEnd - bitStart + 1;
    qaBits = qaBand.rightShift(bitStart).mod(math.pow(2, numBits));
    return qaBits;
  qcMask = extractQABits(modisgppQA, 0, 0).gte(1);
  cloudMask = extractQABits(modisgppQA, 3, 4).gte(1).And(extractQABits(modisgppQA, 3, 4).lt(3));
  qsMask = extractQABits(modisgppQA, 5, 7).gte(2).And(extractQABits(modisgppQA, 5, 7).lt(7));
  mask = qcMask.Or(cloudMask).Or(qsMask);
  return image.updateMask(mask);

In [ ]:
def masklst (image):
  modislstQA = image.select("QC_Day");
  def extractQABits (qaBand, bitStart, bitEnd):
    numBits = bitEnd - bitStart + 1;
    qaBits = qaBand.rightShift(bitStart).mod(math.pow(2, numBits));
    return qaBits;
  mask = extractQABits(modislstQA, 0, 1).gte(1); 
  return image.updateMask(mask);

In [ ]:
def replacegppVals (image):
  currentDate = ee.Date(image.get('system:time_start'));
  meanImage1 =  MODISGPP.filterDate(ee.Date(currentDate).advance(-10,'day'), ee.Date(currentDate).advance(10,'day')).map(maskgpp).mean().select("Gpp");
  meanImage2 =  MODISGPP.filterDate(ee.Date(currentDate).advance(-15,'day'), ee.Date(currentDate).advance(15,'day')).map(maskgpp).mean().select("Gpp");
  meanImage3 =  MODISGPP.filterDate(ee.Date(currentDate).advance(-30,'day'), ee.Date(currentDate).advance(30,'day')).map(maskgpp).mean().select("Gpp");
  meanImage4 =  MODISGPP.filterDate(ee.Date(currentDate).advance(-60,'day'), ee.Date(currentDate).advance(60,'day')).map(maskgpp).mean().select("Gpp")
  return image.unmask(meanImage1).unmask(meanImage2).unmask(meanImage3).unmask(meanImage4)

In [ ]:
def replacelstVals (image):
  currentDate = ee.Date(image.get('system:time_start'));
  meanImage1 =  MODISLST.filterDate(ee.Date(currentDate).advance(-10,'day'), ee.Date(currentDate).advance(10,'day')).map(masklst).mean().select("LST_Day_1km").rename("LST");
  meanImage2 =  MODISLST.filterDate(ee.Date(currentDate).advance(-15,'day'), ee.Date(currentDate).advance(15,'day')).map(masklst).mean().select("LST_Day_1km").rename("LST");
  meanImage3 =  MODISLST.filterDate(ee.Date(currentDate).advance(-30,'day'), ee.Date(currentDate).advance(30,'day')).map(masklst).mean().select("LST_Day_1km").rename("LST");
  meanImage4 =  MODISLST.filterDate(ee.Date(currentDate).advance(-60,'day'), ee.Date(currentDate).advance(60,'day')).map(masklst).mean().select("LST_Day_1km").rename("LST");
  return image.unmask(meanImage1).unmask(meanImage2).unmask(meanImage3).unmask(meanImage4)

# Define time period for prediction and output pathway

In [ ]:
# Define output directory
out_dir = os.path.expanduser('.')
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [ ]:
# Define time period of interests
years = ["2002","2003","2004","2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018","2019"]
months = ["01","02","03","04","05","06","07","08","09","10","11","12"]
days = ["01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]

# Use loop function to automatically extract all covariates for all days within the defined time period for years at smaller properties (smaller than 100 km2)

In [ ]:
# Identify dates with all data available
# The following applies to extraction for years before 2021 because of the current availability of climate data
good_list = [];
for year in years:
  for month in months:
    for day in days:
      try:
        date = year + '-' + month + '-' + day
        ppt_test = precipitation.filterDate(date).size().getInfo();
        Tmin_test = airTmin.filterDate(date).size().getInfo();
        Tmax_test = airTmax.filterDate(date).size().getInfo();
        VPD_test = vpd.filterDate(date).size().getInfo();
        modisndwi_test = NDWI.filterDate(date).size().getInfo();
        modisgpp_test = MODISGPP.filterDate(ee.Date(date).advance(-8,'day'), ee.Date(date).advance(8,'day')).size().getInfo();
        modisevi_test = MODISEVI.filterDate(ee.Date(date)).size().getInfo();    
        modislst_test = MODISLST.filterDate(ee.Date(date)).size().getInfo();         
        modistree_test = MODISTree.filterMetadata("system:index", "contains", str(year)).size().getInfo(); 
        RAP_test = RAP.filterMetadata("system:index", "contains", str(year)).size().getInfo(); 
        if (ppt_test != 0) and (Tmin_test != 0) and (Tmax_test != 0) and (VPD_test != 0) and (modisndwi_test != 0) and (modisgpp_test != 0) and (modisevi_test != 0) \
        and (modislst_test != 0) and (modistree_test != 0) and (RAP_test != 0):
          good_list.append(date)    
      except:
        continue  

In [ ]:
# Identify dates with all data available
# The following applies to extraction for years in and after 2021 because of the current availability of climate data
good_list = [];
for year in years:
  for month in months:
    for day in days:
      try:
        date = year + '-' + month + '-' + day
        ppt_test = precipitation.filter(ee.Filter.date(ee.Date(date).advance(-1,'day'),date)).size().getInfo();
        tmean_test = Tmean.filter(ee.Filter.date(ee.Date(date).advance(-1,'day'),date)).size().getInfo();
        VPDmin_test = vpdmin.filter(ee.Filter.date(ee.Date(date).advance(-1,'day'),date)).size().getInfo();
        VPDmax_test = vpdmax.filter(ee.Filter.date(ee.Date(date).advance(-1,'day'),date)).size().getInfo();
        modisndwi_test = NDWI.filterDate(date).size().getInfo();
        modisgpp_test = MODISGPP.filterDate(ee.Date(date).advance(-8,'day'), ee.Date(date).advance(8,'day')).size().getInfo();
        modisevi_test = MODISEVI.filterDate(ee.Date(date)).size().getInfo();    
        modislst_test = MODISLST.filterDate(ee.Date(date)).size().getInfo();         
        modistree_test = MODISTree.filterMetadata("system:index", "contains", str(2020)).size().getInfo(); 
        RAP_test = RAP.filterMetadata("system:index", "contains", str(2020)).size().getInfo(); 
        if (ppt_test != 0) and (tmean_test != 0) and (VPDmin_test != 0) and (VPDmax_test != 0) and (modisndwi_test != 0) and (modisgpp_test != 0) and (modisevi_test != 0) \
         and (modislst_test != 0) and (modistree_test != 0) and (RAP_test !=0):
          good_list.append(date)    
      except:
        continue  

In [ ]:
# Export covariate images automatically
# The following applies to extraction for years before 2021 
for year in years:
  for month in months:
    for day in days:
      date = year + '-' + month + '-' + day
      if date in good_list:
        ppt = precipitation.filterDate(date).first();
        Tmin = airTmin.filterDate(date).first();
        Tmax = airTmax.filterDate(date).first();
        Tmean = Tmin.add(Tmax).divide(2).rename("tavg");
        VPD = vpd.filterDate(date).first();
        modisndwi = NDWI.filterDate(date).first().rename("ndwi");
        modisgpp = MODISGPP.filterDate(ee.Date(date).advance(-8,'day'), ee.Date(date).advance(8,'day')).map(maskgpp).map(replacegppVals).first().divide(16).select("Gpp");
        modisevi = MODISEVI.filterDate(ee.Date(date)).first().select("EVI");    
        modislst = MODISLST.filterDate(ee.Date(date)).map(masklst).map(replacelstVals).first().select("LST_Day_1km").rename("LST");         
        modistree = MODISTree.filterMetadata("system:index", "contains", str(year)).first().select("Percent_Tree_Cover").rename("tree");                                                                
        tree = RAP.filterMetadata("system:index", "contains", str(year)).first().select("TREE");
        afgc = RAP.filterMetadata("system:index", "contains", str(year)).first().select("AFGC");
        pfgc = RAP.filterMetadata("system:index", "contains", str(year)).first().select("PFGC");
        shb = RAP.filterMetadata("system:index", "contains", str(year)).first().select("SHR");
        bg = RAP.filterMetadata("system:index", "contains", str(year)).first().select("BG");
        ltr = RAP.filterMetadata("system:index", "contains", str(year)).first().select("LTR");
        img = ee.Image.cat(ppt,Tmean,VPD,modisgpp,modisevi,modisndwi,modislst,modistree,tree,afgc,pfgc,shb,bg,ltr) \
                .rename(['ppt','Tavg','VPD','GPP','EVI','NDWI','LST','Tree','TREE','AFGC','PFGC','SHR','BG','LTR'])
        imgname = "cov_" + date +".tif"
        geemap.ee_export_image(img, filename = imgname, scale = 30, region = roi, file_per_band=False)
        !rio cogeo create $imgname cog_$imgname
        !gsutil cp cog_$imgname gs://bucket/Moisture/Test4/"cov_"$date".tif" # Customize the output pathway

In [ ]:
# Export covariate images automatically
# The following applies to extraction for years in and after 2021 
for year in years:
  for month in months:
    for day in days:
      date = year + '-' + month + '-' + day
      if date in good_list:
        ppt = precipitation.filter(ee.Filter.date(ee.Date(date).advance(-1,'day'),date)).first();      
        tmean = Tmean.filter(ee.Filter.date(ee.Date(date).advance(-1,'day'),date)).first();
        VPDmin = vpdmin.filter(ee.Filter.date(ee.Date(date).advance(-1,'day'),date)).first();
        VPDmax = vpdmax.filter(ee.Filter.date(ee.Date(date).advance(-1,'day'),date)).first();
        VPD = VPDmin.add(VPDmax).multiply(100).divide(2).rename("VPD");
        modisndwi = NDWI.filterDate(date).first().rename("ndwi");
        modisgpp = MODISGPP.filterDate(ee.Date(date).advance(-8,'day'), ee.Date(date).advance(8,'day')).map(maskgpp).map(replacegppVals).first().divide(16).select("Gpp");
        modisevi = MODISEVI.filterDate(ee.Date(date)).first().select("EVI");    
        modislst = MODISLST.filterDate(ee.Date(date)).map(masklst).map(replacelstVals).first().select("LST_Day_1km").rename("LST");         
        modistree = MODISTree.filterMetadata("system:index", "contains", str(2020)).first().select("Percent_Tree_Cover").rename("tree");                                                                
        tree = RAP.filterMetadata("system:index", "contains", str(2020)).first().select("TREE");
        afgc = RAP.filterMetadata("system:index", "contains", str(2020)).first().select("AFGC");
        pfgc = RAP.filterMetadata("system:index", "contains", str(2020)).first().select("PFGC");
        shb = RAP.filterMetadata("system:index", "contains", str(2020)).first().select("SHR");
        bg = RAP.filterMetadata("system:index", "contains", str(2020)).first().select("BG");
        ltr = RAP.filterMetadata("system:index", "contains", str(2020)).first().select("LTR");
        img = ee.Image.cat(ppt,tmean,VPD,modisgpp,modisevi,modisndwi,modislst,modistree,tree,afgc,pfgc,shb,bg,ltr) \
                .rename(['ppt','Tavg','VPD','GPP','EVI','NDWI','LST','Tree','TREE','AFGC','PFGC','SHR','BG','LTR'])
        imgname = "cov_" + date +".tif"
        geemap.ee_export_image(img, filename = imgname, scale = 30, region = roi, file_per_band=False)    
        !rio cogeo create $imgname cog_$imgname
        !gsutil cp cog_$imgname gs://bucket/Moisture/Test4/"cov_"$date".tif" # Customize the output pathway

# Use loop function to automatically extract all covariates for all days within the defined time period for years at smaller properties (larger than 100 km2)

In [ ]:
# Export covariate images automatically
# The following applies to extraction for years before 2021 
for year in years:
  for month in months:
    for day in days:
      date = year + '-' + month + '-' + day
      if date in good_list:
        ppt = precipitation.filterDate(date).first();
        Tmin = airTmin.filterDate(date).first();
        Tmax = airTmax.filterDate(date).first();
        Tmean = Tmin.add(Tmax).divide(2).rename("tavg");
        modisndwi = NDWI.filterDate(date).first().rename("ndwi");
        modisgpp = MODISGPP.filterDate(ee.Date(date).advance(-8,'day'), ee.Date(date).advance(8,'day')).map(maskgpp).map(replacegppVals).first().divide(16).select("Gpp");
        modisevi = MODISEVI.filterDate(ee.Date(date)).first().select("EVI");    
        modislst = MODISLST.filterDate(ee.Date(date)).map(masklst).map(replacelstVals).first().select("LST_Day_1km").rename("LST");         
        modistree = MODISTree.filterMetadata("system:index", "contains", str(year)).first().select("Percent_Tree_Cover").rename("tree");                                                                
        tree = RAP.filterMetadata("system:index", "contains", str(year)).first().select("TREE");
        afgc = RAP.filterMetadata("system:index", "contains", str(year)).first().select("AFGC");
        pfgc = RAP.filterMetadata("system:index", "contains", str(year)).first().select("PFGC");
        shb = RAP.filterMetadata("system:index", "contains", str(year)).first().select("SHR");
        bg = RAP.filterMetadata("system:index", "contains", str(year)).first().select("BG");
        ltr = RAP.filterMetadata("system:index", "contains", str(year)).first().select("LTR");
        img = ee.Image.cat(ppt,Tmean,VPD,modisgpp,modisevi,modisndwi,modislst,modistree,tree,afgc,pfgc,shb,bg,ltr) \
                .rename(['ppt','Tavg','VPD','GPP','EVI','NDWI','LST','Tree','TREE','AFGC','PFGC','SHR','BG','LTR'])
        imgname = "cov_" + date +".tif"
        geemap.ee_export_image(ppt, filename = 'ppt.tif', scale = 30, region = roi, file_per_band=False)   
        geemap.ee_export_image(tmean, filename = 'Tavg.tif', scale = 30, region = roi, file_per_band=False)   
        geemap.ee_export_image(VPD, filename = 'VPD.tif', scale = 30, region = roi, file_per_band=False)   
        geemap.ee_export_image(modisgpp, filename = 'GPP.tif', scale = 30, region = roi, file_per_band=False) 
        geemap.ee_export_image(modisevi, filename = 'EVI.tif', scale = 30, region = roi, file_per_band=False)   
        geemap.ee_export_image(modisndwi, filename = 'NDWI.tif', scale = 30, region = roi, file_per_band=False)     
        geemap.ee_export_image(modislst, filename = 'LST.tif', scale = 30, region = roi, file_per_band=False)   
        geemap.ee_export_image(modistree, filename = 'Tree.tif', scale = 30, region = roi, file_per_band=False) 
        geemap.ee_export_image(tree, filename = 'TREE.tif', scale = 30, region = roi, file_per_band=False)   
        geemap.ee_export_image(afgc, filename = 'AFGC.tif', scale = 30, region = roi, file_per_band=False)    
        geemap.ee_export_image(pfgc, filename = 'PFGC.tif', scale = 30, region = roi, file_per_band=False)   
        geemap.ee_export_image(shb, filename = 'SHR.tif', scale = 30, region = roi, file_per_band=False)     
        geemap.ee_export_image(bg, filename = 'BG.tif', scale = 30, region = roi, file_per_band=False)   
        geemap.ee_export_image(ltr, filename = 'LTR.tif', scale = 30, region = roi, file_per_band=False)      
        file_list = ['ppt.tif','Tavg.tif','VPD.tif','GPP.tif','EVI.tif','NDWI.tif','LST.tif','Tree.tif','TREE.tif','AFGC.tif','PFGC.tif','SHR.tif','BG.tif','LTR.tif']
        with rasterio.open(file_list[0]) as src0:
          meta = src0.meta
          meta.update(count = len(file_list))
        with rasterio.open('cov_' + date +'.tif', 'w', **meta) as dst:
          for id, layer in enumerate(file_list, start=1):
            with rasterio.open(layer) as src1:
              dst.write_band(id, src1.read(1))
        !rio cogeo create $imgname cog_$imgname
        !gsutil cp cog_$imgname gs://bucket/Moisture/Test4/"cov_"$date".tif" # Customize the output pathway 
        !rm *.tif

In [ ]:
# Export covariate images automatically
# The following applies to extraction for years in and after 2021 
for year in years:
  for month in months:
    for day in days:
      date = year + '-' + month + '-' + day
      if date in good_list:
        ppt = precipitation.filter(ee.Filter.date(ee.Date(date).advance(-1,'day'),date)).first();      
        tmean = Tmean.filter(ee.Filter.date(ee.Date(date).advance(-1,'day'),date)).first();
        VPDmin = vpdmin.filter(ee.Filter.date(ee.Date(date).advance(-1,'day'),date)).first();
        VPDmax = vpdmax.filter(ee.Filter.date(ee.Date(date).advance(-1,'day'),date)).first();
        VPD = VPDmin.add(VPDmax).multiply(100).divide(2).rename("VPD");
        modisndwi = NDWI.filterDate(date).first().rename("ndwi");
        modisgpp = MODISGPP.filterDate(ee.Date(date).advance(-8,'day'), ee.Date(date).advance(8,'day')).map(maskgpp).map(replacegppVals).first().divide(16).select("Gpp");
        modisevi = MODISEVI.filterDate(ee.Date(date)).first().select("EVI");    
        modislst = MODISLST.filterDate(ee.Date(date)).map(masklst).map(replacelstVals).first().select("LST_Day_1km").rename("LST");         
        modistree = MODISTree.filterMetadata("system:index", "contains", str(2020)).first().select("Percent_Tree_Cover").rename("tree");                                                                
        tree = RAP.filterMetadata("system:index", "contains", str(2020)).first().select("TREE");
        afgc = RAP.filterMetadata("system:index", "contains", str(2020)).first().select("AFGC");
        pfgc = RAP.filterMetadata("system:index", "contains", str(2020)).first().select("PFGC");
        shb = RAP.filterMetadata("system:index", "contains", str(2020)).first().select("SHR");
        bg = RAP.filterMetadata("system:index", "contains", str(2020)).first().select("BG");
        ltr = RAP.filterMetadata("system:index", "contains", str(2020)).first().select("LTR");
        img = ee.Image.cat(ppt,tmean,VPD,modisgpp,modisevi,modisndwi,modislst,modistree,tree,afgc,pfgc,shb,bg,ltr) \
                .rename(['ppt','Tavg', 'VPD', 'GPP', 'EVI', 'NDWI', 'LST', 'Tree', 'TREE', 'AFGC', 'PFGC', 'SHR', 'BG', 'LTR'])
        imgname = "cov_" + date +".tif"
        geemap.ee_export_image(ppt, filename = 'ppt.tif', scale = 30, region = roi, file_per_band=False)   
        geemap.ee_export_image(tmean, filename = 'Tavg.tif', scale = 30, region = roi, file_per_band=False)   
        geemap.ee_export_image(VPD, filename = 'VPD.tif', scale = 30, region = roi, file_per_band=False)   
        geemap.ee_export_image(modisgpp, filename = 'GPP.tif', scale = 30, region = roi, file_per_band=False) 
        geemap.ee_export_image(modisevi, filename = 'EVI.tif', scale = 30, region = roi, file_per_band=False)   
        geemap.ee_export_image(modisndwi, filename = 'NDWI.tif', scale = 30, region = roi, file_per_band=False)     
        geemap.ee_export_image(modislst, filename = 'LST.tif', scale = 30, region = roi, file_per_band=False)   
        geemap.ee_export_image(modistree, filename = 'Tree.tif', scale = 30, region = roi, file_per_band=False) 
        geemap.ee_export_image(tree, filename = 'TREE.tif', scale = 30, region = roi, file_per_band=False)   
        geemap.ee_export_image(afgc, filename = 'AFGC.tif', scale = 30, region = roi, file_per_band=False)    
        geemap.ee_export_image(pfgc, filename = 'PFGC.tif', scale = 30, region = roi, file_per_band=False)   
        geemap.ee_export_image(shb, filename = 'SHR.tif', scale = 30, region = roi, file_per_band=False)     
        geemap.ee_export_image(bg, filename = 'BG.tif', scale = 30, region = roi, file_per_band=False)   
        geemap.ee_export_image(ltr, filename = 'LTR.tif', scale = 30, region = roi, file_per_band=False)      
        file_list = ['ppt.tif','Tavg.tif','VPD.tif','GPP.tif','EVI.tif','NDWI.tif','LST.tif','Tree.tif','TREE.tif','AFGC.tif','PFGC.tif','SHR.tif','BG.tif','LTR.tif']
        with rasterio.open(file_list[0]) as src0:
          meta = src0.meta
          meta.update(count = len(file_list))
        with rasterio.open('cov_' + date +'.tif', 'w', **meta) as dst:
          for id, layer in enumerate(file_list, start=1):
            with rasterio.open(layer) as src1:
              dst.write_band(id, src1.read(1))
        !rio cogeo create $imgname cog_$imgname
        !gsutil cp cog_$imgname gs://bucket/Moisture/Test4/"cov_"$date".tif" # Customize the output pathway
        !rm *.tif

# Extract NLDAS covariate for all days within the defined time period

In [ ]:
NLDAS = ee.ImageCollection("projects/pathway") # Use pathway for where NLDAS data was deposited from the previous step

In [ ]:
# Generate dates with available data
good_list = [];
for year in years:
  for month in months:
    for day in days:
      try:
        date = year + '-' + month + '-' + day
        NLDAS_test = NLDAS.filterDate(date).size().getInfo(); 
        if (NLDAS_test != 0):
          good_list.append(date)    
      except:
        continue  

In [ ]:
# Run model in the loop in order to extract data from three depth layers (can be customized) automatically
for year in years:
  for month in months:
    for day in days:
      date = year + '-' + month + '-' + day
      if date in good_list:
        img = NLDAS.filterDate(date).first().select(['B0','B1','B3']);                                                                
        imgname = "nldas_" + date +".tif"
        geemap.ee_export_image(img, filename = imgname, scale = 30, region = roi, file_per_band=False)
        !rio cogeo create $imgname cog_$imgname
        !gsutil cp cog_$imgname gs://bucket/Moisture/Test4/"nldas_"$date".tif" # Customize the output pathway

# Check missing files

In [ ]:
#The following example checks data from one specific year
covfiles = !gsutil ls gs://bucket/Moisture/Test4/cov_2019*tif # Costumize the input pathway

In [ ]:
import datetime

In [ ]:
start = datetime.date(2019, 1, 1)
end = datetime.date(2019, 12, 31)

In [ ]:
days = [start + datetime.timedelta(days=x) for x in range((end-start).days + 1)]

In [ ]:
daylist = []
for day in days:
    day = day.strftime('%Y-%m-%d')
    daylist.append(day)

In [ ]:
covdate_list = []
for filename in covfiles:
  covfile_name = filename.split('_')[-1]
  covfile_name = covfile_name.split('.')[-2]
  covdate_list.append(covfile_name)

In [ ]:
bad_list = list(set(daylist)-set(covdate_list))

In [ ]:
bad_list

In [ ]:
nldasfiles = !gsutil ls gs://bucket/Moisture/Test4/nldas_2019*tif # Costumize the input pathway

In [ ]:
nldasdate_list = []
for filename in nldasfiles:
  nldasfile_name = filename.split('_')[-1]
  nldasfile_name = nldasfile_name.split('.')[-2]
  nldasdate_list.append(nldasfile_name)

In [ ]:
bad_nldas_list = list(set(daylist)-set(nldasdate_list))

In [ ]:
bad_nldas_list 

# Generate replacement for days missing covariates using values averaged over longer period of time (using climate data as an example)

In [ ]:
climate_bad_list = []
ndwi_bad_list = []
gpp_bad_list = []
evi_bad_list = []
lst_bad_list = []
tree_bad_list = []
rap_bad_list = []
for year in years:
  for month in months:
    for day in days:
      date = year + '-' + month + '-' + day
      if date in bad_list:
        date = year + '-' + month + '-' + day
        ppt_test = precipitation.filterDate(date).size().getInfo();
        Tmin_test = airTmin.filterDate(date).size().getInfo();
        Tmax_test = airTmax.filterDate(date).size().getInfo();
        VPD_test = vpd.filterDate(date).size().getInfo();
        modisndwi_test = NDWI.filterDate(date).size().getInfo();
        modisgpp_test = MODISGPP.filterDate(ee.Date(date).advance(-8,'day'), ee.Date(date).advance(8,'day')).size().getInfo();
        modisevi_test = MODISEVI.filterDate(ee.Date(date)).size().getInfo();   
        modislst_test = MODISLST.filterDate(ee.Date(date)).size().getInfo();         
        modistree_test = MODISTree.filterMetadata("system:index", "contains", str(year)).size().getInfo(); 
        RAP_test = RAP.filterMetadata("system:index", "contains", str(year)).size().getInfo();
        if (ppt_test == 0) | (Tmin_test == 0) | (Tmax_test == 0) | (VPD_test == 0):
          climate_bad_list.append(date)
        if (modisndwi_test == 0):
          ndwi_bad_list.append(date)
        if (modisgpp_test == 0):
          gpp_bad_list.append(date)
        if (modisevi_test == 0):
          evi_bad_list.append(date)
        if (modislst_test == 0):
          lst_bad_list.append(date)
        if (modistree_test == 0):
          tree_bad_list.append(date)
        if (RAP_test == 0):
          rap_bad_list.append(date)

In [ ]:
climate_bad_list
#ndwi_bad_list
#gpp_bad_list
#evi_bad_list
#lst_bad_list
#tree_bad_list
#rap_bad_list

In [ ]:
for year in years:
  for month in months:
    for day in days:
      date = year + '-' + month + '-' + day
      if date in climate_bad_list:
        ppt = precipitation.filterDate(ee.Date(date).advance(-3,'day'), ee.Date(date).advance(3,'day')).mean();
        Tmin = airTmin.filterDate(ee.Date(date).advance(-3,'day'), ee.Date(date).advance(3,'day')).mean();
        Tmax = airTmax.filterDate(ee.Date(date).advance(-3,'day'), ee.Date(date).advance(3,'day')).mean();
        Tmean = Tmin.add(Tmax).divide(2).rename("tavg");
        VPD = vpd.filterDate(ee.Date(date).advance(-3,'day'), ee.Date(date).advance(3,'day')).mean();
        modisndwi = NDWI.filterDate(date).first().rename("ndwi");
        modisgpp = MODISGPP.filterDate(ee.Date(date).advance(-8,'day'), ee.Date(date).advance(8,'day')).map(maskgpp).map(replacegppVals).first().divide(16).select("Gpp");
        modisevi = MODISEVI.filterDate(ee.Date(date)).first().select("EVI"); 
        modislst = MODISLST.filterDate(ee.Date(date)).map(masklst).map(replacelstVals).first().select("LST_Day_1km").rename("LST");         
        modistree = MODISTree.filterMetadata("system:index", "contains", str(year)).first().select("Percent_Tree_Cover").rename("tree"); 
        tree = RAP.filterMetadata("system:index", "contains", str(year)).first().select("TREE");                                                               
        afgc = RAP.filterMetadata("system:index", "contains", str(year)).first().select("AFGC");
        pfgc = RAP.filterMetadata("system:index", "contains", str(year)).first().select("PFGC");
        shb = RAP.filterMetadata("system:index", "contains", str(year)).first().select("SHR");
        bg = RAP.filterMetadata("system:index", "contains", str(year)).first().select("BG");
        ltr = RAP.filterMetadata("system:index", "contains", str(year)).first().select("LTR");
        img = ee.Image.cat(ppt,Tmean,VPD,modisgpp,modisevi,modisndwi,modislst,modistree,tree,afgc,pfgc,shb,bg,ltr) \
                .rename(['ppt','Tavg','VPD','GPP','EVI','NDWI','LST','Tree','TREE','AFGC','PFGC','SHR','BG','LTR'])
        imgname = "cov_" + date +".tif"
        geemap.ee_export_image(img, filename = imgname, scale = 30, region = roi, file_per_band=False)
        !rio cogeo create $imgname cog_$imgname
        !gsutil cp cog_$imgname gs://bucket/Moisture/Test4/"cov_"$date".tif" # Customize the output pathway

# Generate replacement for days missing NLDAS using values averaged over longer period of time

In [ ]:
for year in years:
  for month in months:
    for day in days:
      date = year + '-' + month + '-' + day
      if date in bad_nldas_list:
        img = NLDAS.filterDate(ee.Date(date).advance(-1,'day'), ee.Date(date).advance(1,'day')).mean().select(['B0','B1','B3']);                                                                
        imgname = "nldas_" + date +".tif"
        geemap.ee_export_image(img, filename = imgname, scale = 30, region = roi, file_per_band=False)
        !rio cogeo create $imgname cog_$imgname
        !gsutil cp cog_$imgname gs://bucket/Moisture/Test4/"nldas_"$date".tif" # Customize the output pathway